In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
import json
import configparser
from pathlib import Path
import os

from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.metrics import mean_squared_error as MSE
def RMSE(y_real, y_predict):
    return np.sqrt(MSE(y_real, y_predict))
    
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling1D

configs = configparser.ConfigParser()
configs.read("configuracoes.conf",encoding='utf-8')

data_folder = Path(configs["DEFAULT"]["folder_path"])
path = data_folder / configs["DEFAULT"]["file"]
objeto = configs["DEFAULT"]["objeto"]
regiao = configs["DEFAULT"]["regiao"]

In [2]:
look_back = 12 #Quantidade de meses anteriores usado para prever
forecast_horizon = int(configs["DEFAULT"]["meses"]) #Quantidade de meses previstos

model_mape = []
model_rmse = []

for i in range(0,1):

    #Carrega os dados
    cesta = pd.read_excel(path)
    cesta.drop(['ano', 'salario', 'preco'], axis=1, inplace=True)

    #Cria sequencia de dados do tamanho (meses anterioes + meses a serem previstos)
    for n_step in range(1, look_back + forecast_horizon):
        cesta[f'razao t(h + {n_step})'] = cesta['razao'].shift(-n_step).values
    cesta.dropna(inplace=True)
    cesta.reset_index(drop=True, inplace=True)

    #Divide os dados em Treino e Teste
    #Validação
    #X_train = cesta.iloc[:-68:, :look_back:].values
    #y_train = cesta.iloc[:-68:, look_back::].values
    #X_val = cesta.iloc[-68:-38:, :look_back:].values
    #y_val = cesta.iloc[-68:-38:, look_back::].values
    #y_val2 = cesta.iloc[-68:-38:, look_back::].values.tolist()
    
    #Teste
    #X_train = cesta.iloc[:-38:, :look_back:].values
    #y_train = cesta.iloc[:-38:, look_back::].values
    #X_val = cesta.iloc[-38::, :look_back:].values
    #y_val = cesta.iloc[-38::, look_back::].values
    #y_val2 = cesta.iloc[-38::, look_back::].values.tolist()
    
    #6 Meses
    X_train = cesta.iloc[:-1:, :look_back:].values
    y_train = cesta.iloc[:-1:, look_back::].values
    X_val = cesta.iloc[-1::, :look_back:].values
    y_val = cesta.iloc[-1::, look_back::].values
    y_val2 = cesta.iloc[-1::, look_back::].values.tolist()

    #remodela os dados para formato (batchs, months, features)
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

    #Cria o modelo
    model=Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, input_shape=(look_back,1), activation='tanh'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(25, activation='tanh'))
    model.add(Dense(forecast_horizon))
    model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.00003))

    #Treina e pever (forecast_horizon) quantidade de tempo
    history = model.fit(X_train, y_train, epochs= 70,  batch_size=1, shuffle=False)
    forecast = model.predict(X_val)

    model_rmse.append(RMSE(y_val2, forecast))
    model_mape.append(MAPE(y_val2, forecast) * 100)

""" model_mape.sort()
model_rmse.sort()
model_mape.pop()
del model_mape[0]
model_rmse.pop()
del model_rmse[0]

mean_rmse = 0
mean_mape = 0

for rmse, mape in zip(model_rmse, model_mape):
    mean_rmse += rmse
    mean_mape += mape

mean_rmse = mean_rmse / 5
mean_mape = mean_mape / 5 """

resultados = [valor for valor in forecast[0]]
objeto = objeto.replace(" ",'_')
output = {objeto.lower():str(resultados)}
filename = f"previsao_CNN_{objeto.lower()}_{regiao.lower()}_{forecast_horizon}_meses.json"
pasta_atual = os.getcwd()
full_url = Path(pasta_atual) / 'resultados' / 'previsoes' / filename
os.makedirs(os.path.dirname(full_url), exist_ok=True)
with open(full_url,"w") as file:
    json.dump(output,file,ensure_ascii=False)
print("Arquivo com resultados gerados")


Epoch 1/70
189/189 [==============================] - 0s 902us/step - loss: 0.0169
Epoch 2/70
189/189 [==============================] - 0s 660us/step - loss: 0.0011
Epoch 3/70
189/189 [==============================] - 0s 686us/step - loss: 0.0012
Epoch 4/70
189/189 [==============================] - 0s 665us/step - loss: 0.0012
Epoch 5/70
189/189 [==============================] - 0s 945us/step - loss: 0.0012
Epoch 6/70
189/189 [==============================] - 0s 792us/step - loss: 0.0012
Epoch 7/70
189/189 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 8/70
189/189 [==============================] - 0s 712us/step - loss: 0.0012
Epoch 9/70
189/189 [==============================] - 0s 789us/step - loss: 0.0012
Epoch 10/70
189/189 [==============================] - 0s 776us/step - loss: 0.0012
Epoch 11/70
189/189 [==============================] - 0s 686us/step - loss: 0.0012
Epoch 12/70
189/189 [==============================] - 0s 755us/step - loss: 0.0012
Epo

In [3]:
#print("MAPE = ", round(mean_mape, 3))
#print("RMSE = ", round(mean_rmse, 5))
print("MAPE = ", round(model_mape[0], 3))
print("RMSE = ", round(model_rmse[0], 5))

MAPE =  5.921
RMSE =  0.02676
